In [ ]:
import os

import pickle

import StaticWindow as sw

from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Load in Data

In [ ]:
# Load event data generated by StaticWindow.ipynb
directory = 'INSERT DIRECTORY'
with open(os.path.join(directory, 'INSERT FILENAME'), 'rb') as f:
    evt_df = pickle.load(f)

In [ ]:
traces_shape = (10, 1000000) # Size of the simulated data matrix

# Generate Predictions
Convert the saved event locations to one hot encodings

In [ ]:
def my_func(row):
    
    preds = []
    for dev in row.iteritems():
        pred = sw.locs_to_preds(dev[1], traces_shape)
        preds.append(pred)
    return preds

evt_df['predictions'] = evt_df.apply(my_func)

In [ ]:
evt_df.head()

# Load in Data

In [ ]:
directory = 'INSERT DIRECTORY' # Directory where events generated by Simulation.py are saved.

files = [] # List of simulation filenames

SNRs = [float(x.split('_')[0]) for x in files]

# Load the ground truth anomalies
_, ground_truths = sw.load_SNR_data(directory, files)

# Tidy Data

In [ ]:
truth_df = pd.Series(ground_truths).to_frame()
truth_df.columns = ['ground_truths']
truth_df.index.names = ['SNR']
truth_df.head()

# Score Predictions

In [ ]:
# Create new DataFrame to store scores
columns = ['SNR', 'Thresh', 'trace_idx', 'FPR', 'TPR', 'FNR', 'TNR']
result_df = pd.DataFrame(columns=columns)
#result_df = result_df.set_index(['SNR', 'St_Dev'])
result_df.head()

In [ ]:
# Initialise window size and step size
window_size = 10
step_size = 1

# Check for missed trace points
untested = (1000000 - (window_size - step_size)) % step_size
untested

In [ ]:
for snr, thresh in list(evt_df.index.unique()):
    print('SNR: ' + str(snr) + ', Threshold: ' + str(thresh))
    preds = evt_df.loc[snr, thresh]['predictions'] # Get predictions
    truths = truth_df.loc[snr][0] # Get ground truth
    
    assert preds.shape == truths.shape
    for i in range(preds.shape[0]):
        trace_idx = i
        
        # Generate detection scores
        C = confusion_matrix(truths[i], preds[i], normalize='true')
        # C = confusion_matrix(truths[i][:-untested], preds[i][:-untested], normalize='true')
        tpr = C[1, 1]
        fpr = C[0, 1]
        tnr = C[0, 0]
        fnr = C[1, 0]
        
        # Populate DataFrame
        entry = dict(zip( columns, [[snr], [thresh], [trace_idx], [fpr], [tpr], [fnr], [tnr]]))                 
        entry_df = pd.DataFrame(entry)
        
        result_df = pd.concat([result_df, entry_df])

# Tidy DataFrame
result_df = result_df.set_index(['SNR', 'Thresh'])        
result_df.head()

# Save Results

In [ ]:
with open('INSERT FILENAME', 'wb') as f:
    pickle.dump(result_df, f)